# Explainability Analysis

This notebook performs explainability analysis using LIME on the best-performing model:  

Feature Selection: **Mutual Information Regression (MIR)**  
Model: **Logistic Regression**

In [ ]:
import os
import joblib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from lime.lime_tabular import LimeTabularExplainer
from sklearn.preprocessing import StandardScaler

DATA_DIR = "../data/processed/features2/mir"
MODEL_PATH = "../models/ml2/mir/logistic_regression.pkl"

OUT_FIG_DIR = "../figures/xai"
OUT_CSV_DIR = "../data/processed/xai"

os.makedirs(OUT_FIG_DIR, exist_ok=True)
os.makedirs(OUT_CSV_DIR, exist_ok=True)

## Load Dataset & Model

In [ ]:
train_df = pd.read_csv(f"{DATA_DIR}/train.csv")
test_df = pd.read_csv(f"{DATA_DIR}/test.csv")

model = joblib.load(MODEL_PATH)

TARGET = "DepressionEncoded"

X_train = train_df.drop(columns=[TARGET])
y_train = train_df[TARGET]

X_test = test_df.drop(columns=[TARGET])
y_test = test_df[TARGET]

print("Training Shape:", X_train.shape)
print("Test Shape:", X_test.shape)

## Feature Names & Labels

In [ ]:
feature_names = X_train.columns.tolist()

class_names = [
    "Minimal",
    "Mild",
    "Moderate",
    "Moderately Severe",
    "Severe"
]

## Standardization

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

## Initialize LIME Explainer

In [ ]:
explainer = LimeTabularExplainer(
    X_train_scaled,
    feature_names=feature_names,
    class_names=class_names,
    discretize_continuous=True,
    mode="classification"
)

## Select Exactly One Sample Per Class

In [ ]:
selected_samples = {}

for class_id in range(5):
    idx = y_test[y_test == class_id].index[0]
    selected_samples[class_id] = X_test.loc[idx].values

selected_samples

## Run LIME Explainability for Each Class

In [ ]:
summary_rows = []

for class_id, instance in selected_samples.items():
    
    instance_scaled = scaler.transform(instance.reshape(1, -1))[0]

    exp = explainer.explain_instance(
        instance_scaled,
        model.predict_proba,
        num_features=13
    )

    class_name = class_names[class_id]

    html_path = f"{OUT_CSV_DIR}/lime_{class_name.lower().replace(' ', '_')}_{class_id}.html"
    exp.save_to_file(html_path)

    weights = pd.DataFrame(exp.as_list(), columns=["Feature", "Weight"])
    csv_path = f"{OUT_CSV_DIR}/lime_{class_name.lower().replace(' ', '_')}_{class_id}_weights.csv"
    weights.to_csv(csv_path, index=False)

    plt.figure(figsize=(9, 5))

    weights_sorted = weights.sort_values(by="Weight")

    colors = ["green" if val > 0 else "red" for val in weights_sorted["Weight"]]

    plt.barh(
        weights_sorted["Feature"],
        weights_sorted["Weight"],
        color=colors
    )

    plt.axvline(0, color="black", linewidth=1)
    plt.title(f"LIME Explanation - {class_name}")
    plt.xlabel("Feature Contribution")
    plt.ylabel("Feature")
    plt.tight_layout()

    png_path = f"{OUT_FIG_DIR}/lime_{class_name.lower().replace(' ', '_')}_{class_id}.png"
    plt.savefig(png_path)
    plt.show()
    plt.close()


    weights["Class"] = class_name
    summary_rows.append(weights)

    print(f"✅ LIME completed for class: {class_name}")

## Save Global Summary Across All Classes

In [ ]:
lime_summary = pd.concat(summary_rows)

summary_path = f"{OUT_CSV_DIR}/lime_mir_summary.csv"
lime_summary.to_csv(summary_path, index=False)

lime_summary.head()